In [103]:
import pandas as pd
import os
import yaml

In [104]:
table = pd.read_csv("./xQTL conda packages - Sheet1 (2).csv")

In [105]:
table = table[table.Channel.notna()]

In [106]:
table

,Package,URL,Language,Version,Git revision,Channel,Environment,License,Conda package,Notes,Docker image name,Universial Package
0,METAL,https://github.com/statgen/METAL,C/C++,2020_05_05,NaN,dnachun,METAL,?,https://anaconda.org/dnachun/metal,NaN,NaN,NaN
1,MOFA2,https://bioconductor.org/packages/3.16/bioc/ht...,R/Python,1.10.0,NaN,bioconda,TensorQTL,LGPL v3,https://anaconda.org/bioconda/bioconductor-mofa2,bioconda package is outdated,NaN,NaN
2,plink,https://www.cog-genomics.org/plink2,C/C++,1.90b6.21,NaN,bioconda,"TensorQTL, bioinfo",GPL,https://anaconda.org/bioconda/plink,NaN,NaN,NaN
3,qvalue,https://www.bioconductor.org/packages/release/...,R,2.32.0,NaN,bioconda,"TensorQTL, qvalue",NaN,https://anaconda.org/bioconda/bioconductor-qvalue,NaN,NaN,NaN
4,multipy,https://github.com/puolival/multipy,Python,0.16,NaN,dnachun,TensorQTL,NaN,https://anaconda.org/dnachun/multipy,Git was used because a fix was needed to suppo...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
69,susif,https://github.com/stephenslab/susiF.alpha,R,0.1.212,NaN,dnachun,stephenslab,NaN,https://anaconda.org/dnachun/r-susif.alpha,NaN,NaN,NaN
70,mvf.susie,https://github.com/william-denault/mvf.susie.a...,R,0.1.021,NaN,dnachun,stephenslab,NaN,https://anaconda.org/dnachun/r-mvf.susie.alpha,NaN,NaN,NaN
71,tidyverse,NaN,R,NaN,NaN,conda-forge,Universial,NaN,https://anaconda.org/conda-forge/r-tidyverse,NaN,NaN,NaN
72,pandas,NaN,Python,NaN,NaN,conda-forge,Universial,NaN,https://anaconda.org/conda-forge/pandas,NaN,NaN,NaN


In [107]:
yml_tempt = dict()

In [108]:
yml_tempt["name"]  = "stephenslab"
yml_tempt["channels"] = list(set(table.query("Environment == 'stephenslab' | Environment == 'Universial'").Channel))
yml_tempt["dependencies"] = [ x.split("/")[-1] for x in  set(table.query("Environment == 'stephenslab' | Environment == 'Universial'").query("Channel in @yml_tempt['channels'] "  ).iloc[:,8] )]
yml_tempt["channels"].sort(reverse=True)
yml_tempt["channels"].append("nodefaults")

In [78]:
yml_tempt["dependency"]

['https://anaconda.org/dnachun/r-flashier',
 'https://anaconda.org/dnachun/r-mvsusier',
 'https://anaconda.org/dnachun/r-gwasrapidd',
 'https://anaconda.org/dnachun/r-mashr',
 'https://anaconda.org/bioconda/bioconductor-variantannotation',
 'https://anaconda.org/conda-forge/r-rfast',
 'https://anaconda.org/dnachun/r-flashr',
 'https://anaconda.org/dnachun/r-mvf.susie.alpha',
 'https://anaconda.org/bioconda/r-coloc',
 'https://anaconda.org/conda-forge/r-tidyverse',
 'https://anaconda.org/dnachun/r-udr',
 'https://anaconda.org/dnachun/r-susif.alpha',
 'https://anaconda.org/conda-forge/backports.zoneinfo',
 'https://anaconda.org/dnachun/python-ldstore',
 'https://anaconda.org/conda-forge/r-cowplot',
 'https://anaconda.org/conda-forge/r-complexupset',
 'https://anaconda.org/conda-forge/pandas',
 'https://anaconda.org/conda-forge/r-susier']

In [111]:
yml_tempt

{'name': 'stephenslab',
 'channels': ['dnachun', 'conda-forge', 'bioconda', 'nodefaults'],
 'dependency': ['r-flashier',
  'r-mvsusier',
  'r-gwasrapidd',
  'r-mashr',
  'bioconductor-variantannotation',
  'r-rfast',
  'r-flashr',
  'r-mvf.susie.alpha',
  'r-coloc',
  'r-tidyverse',
  'r-udr',
  'r-susif.alpha',
  'backports.zoneinfo',
  'python-ldstore',
  'r-cowplot',
  'r-complexupset',
  'pandas',
  'r-susier']}

In [112]:
class IndentDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(IndentDumper, self).increase_indent(flow, False)

In [113]:
with open("test.yml", 'w') as f:
        yaml.dump(yml_tempt,f,sort_keys = False, Dumper=IndentDumper )

In [85]:
sorted(yml_tempt["channels"]).reverse()

In [89]:
yml_tempt["channels"].sort(reverse=True)

In [88]:
reverse(sorted(yml_tempt["channels"]))

NameError: name 'reverse' is not defined

In [90]:
yml_tempt["channels"]

['nodefaults', 'dnachun', 'conda-forge', 'bioconda']

In [115]:
print(yaml.dump(yml_tempt,sort_keys = False, Dumper=IndentDumper ))

name: stephenslab
channels:
  - dnachun
  - conda-forge
  - bioconda
  - nodefaults
dependency:
  - r-flashier
  - r-mvsusier
  - r-gwasrapidd
  - r-mashr
  - bioconductor-variantannotation
  - r-rfast
  - r-flashr
  - r-mvf.susie.alpha
  - r-coloc
  - r-tidyverse
  - r-udr
  - r-susif.alpha
  - backports.zoneinfo
  - python-ldstore
  - r-cowplot
  - r-complexupset
  - pandas
  - r-susier

